In [1]:
## 2. tensorflow-gpu测试代码

In [ ]:
import tensorflow as tf

print(tf.__version__)


In [ ]:
import tensorflow as tf
hello = tf.constant('Hello, Word!')  
sess = tf.Session()  
print(sess.run(hello))  

In [ ]:
# cpu+gpu运行

import tensorflow as tf

with tf.device('/cpu:0'):
    a = tf.constant([1.0,2.0,3.0],shape=[3],name='a')
    b = tf.constant([1.0,2.0,3.0],shape=[3],name='b')
with tf.device('/gpu:0'):
    c = a+b
   
#注意：allow_soft_placement=True表明：计算设备可自行选择，如果没有这个参数，会报错。
#因为不是所有的操作都可以被放在GPU上，如果强行将无法放在GPU上的操作指定到GPU上，将会报错。

sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,log_device_placement=True))
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(tf.global_variables_initializer())
print(sess.run(c))

In [ ]:
# linux下指定GPU

!CUDA_VISIBLE_DEVICES=0 python gpu.py gpu 1500

# !CUDA_VISIBLE_DEVICES=0 python tensorflow/examples/tutorials/mnist/mnist_with_summaries.py


## 性能测试

可以使用CPU或GPU分别计算，比较性能。

In [2]:
import sys
import numpy as np
import tensorflow as tf
from datetime import datetime

# 设置为gpu或cpu分别执行
device_name="/cpu:0" 
 
shape=(int(10000),int(10000))
 
with tf.device(device_name):
    #形状为shap,元素服从minval和maxval之间的均匀分布
    random_matrix = tf.random_uniform(shape=shape, minval=0, maxval=1)
    dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
    sum_operation = tf.reduce_sum(dot_operation)
 
startTime = datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
    result = session.run(sum_operation)
print(result)
 
print("\n" * 2)
print("Shape:", shape, "Device:", device_name)
print("Time taken:", datetime.now() - startTime)

254430800000.0



Shape: (10000, 10000) Device: /cpu:0
Time taken: 0:00:19.444104


## 查看GPU内存使用

In [23]:
import pynvml

pynvml.nvmlInit()

# 显示驱动信息
print("GPU Driver: %s" % pynvml.nvmlSystemGetDriverVersion())

# 查询GPU内存信息
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
print('Memory total:%sM, used:%sM, free:%sM' % (int(meminfo.total/1024**2), int(meminfo.used/1024**2), int(meminfo.free/1024**2))) 

# 查询GPU显卡型号
deviceCount = pynvml.nvmlDeviceGetCount()#几块显卡
for i in range(deviceCount):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)

    print("Device", i, ":", pynvml.nvmlDeviceGetName(handle)) 
    print("Temperature is %d C"%pynvml.nvmlDeviceGetTemperature(handle,0))
    print("Power ststus",pynvml.nvmlDeviceGetPowerState(handle))
    
pynvml.nvmlShutdown()

GPU Driver: b'418.39'
Memory total:4040M, used:558M, free:3481M
Device 0 : b'GeForce GTX 1050 Ti'
Temperature is 55 C
Power ststus 0


### G7-7588 显卡信息


GeForce GTX 1050 Ti 
Pascal 768 Up to 4 GB GDDR5

---------------------
GeForce GTX 1050
Pascal 640Up to 4 GB GDDR5

In [ ]:
def is_gpu_available(cuda_only=True):
  """
  code from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/platform/test.py
  Returns whether TensorFlow can access a GPU.
  Args:
    cuda_only: limit the search to CUDA gpus.
  Returns:
    True iff a gpu device of the requested kind is available.
  """
  from tensorflow.python.client import device_lib as _device_lib

  if cuda_only:
    return any((x.device_type == 'GPU')
               for x in _device_lib.list_local_devices())
  else:
    return any((x.device_type == 'GPU' or x.device_type == 'SYCL')
               for x in _device_lib.list_local_devices())

def get_available_gpus():
    """
    code from http://stackoverflow.com/questions/38559755/how-to-get-current-available-gpus-in-tensorflow
    """
    from tensorflow.python.client import device_lib as _device_lib
    local_device_protos = _device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def test_is_gpu_available():
    #for i in range(4):
    if(is_gpu_available()):
        str = 'with' if is_gpu_available(True) else 'without'
        print("GPU is available, %s CUDA installed" % str)

def test_get_available_gpus():
    devices = get_available_gpus();
    i = 0 
    for d in devices:
        i = i + 1
        print("你的第 %2d 个GPU是： %5s" % (i,d))
        
test_is_gpu_available()

test_get_available_gpus()


In [ ]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


In [ ]:
# -*- coding: utf-8 -*-
"""
    测试NVIDIA cuda。
"""

import tensorflow as tf
import numpy as np
import time

value = np.random.randn(5000,1000)
a = tf.constant(value)

b = a*a

tic = time.time()
with tf.Session() as sess:
    for i in range(1000):
        sess.run(b)
        
toc = time.time()
t_cost = toc - tic

print(t_cost)

In [ ]:
import tensorflow as tf

a = tf.constant([1.0, 2.0, 3.0], shape=[3], name='a')
b = tf.constant([1.0, 2.0, 3.0], shape=[3], name='b')
c = a + b
# 通过log_device_placement参数来输出运行每一个运算的设备。
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

In [ ]:
import tensorflow as tf

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)) as sess:
    a = tf.constant(1)
    b = tf.constant(3)
    c = a + b
    print('结果是：%d\n 值为：%d' % (sess.run(c), sess.run(c)))


!CUDA_VISIBLE_DEVICES=0 python gpu.py gpu 1500

Environment Variable Syntax      Results

    CUDA_VISIBLE_DEVICES=1           Only device 1 will be seen
    CUDA_VISIBLE_DEVICES=0,1         Devices 0 and 1 will be visible
    CUDA_VISIBLE_DEVICES="0,1"       Same as above, quotation marks are optional
    CUDA_VISIBLE_DEVICES=0,2,3       Devices 0, 2, 3 will be visible; device 1 is masked
    CUDA_VISIBLE_DEVICES=""          No GPU will be visible

In [ ]:
# 指定使用的GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))        

In [ ]:
import sys
import numpy as np
import tensorflow as tf
from datetime import datetime


#device_name = sys.argv[1]  # Choose device from cmd line. Options: gpu or cpu
device_name = 'gpu'
#shape = (int(sys.argv[2]), int(sys.argv[2]))
shape = (1500,1500)
if device_name == "gpu":
    device_name = "/gpu:0"
else:
    device_name = "/cpu:0"

with tf.device(device_name):
    random_matrix = tf.random_uniform(shape=shape, minval=0, maxval=1)
    dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
    sum_operation = tf.reduce_sum(dot_operation)


startTime = datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
        result = session.run(sum_operation)
        print(result)

# It can be hard to see the results on the terminal with lots of output -- add some newlines to improve readability.
print("\n" * 5)
print("Shape:", shape, "Device:", device_name)
print("Time taken:", datetime.now() - startTime)

print("\n" * 5)

In [ ]:
import numpy as np 
from timeit import default_timer as timer
from numba import vectorize

@vectorize(["float32(float32, float32)"], target='cuda')
def vectorAdd(a, b):
    return a + b

def main():
    N = 320000000

    A = np.ones(N, dtype=np.float32 )
    B = np.ones(N, dtype=np.float32 )
    C = np.zeros(N, dtype=np.float32 )

    start = timer()
    C = vectorAdd(A, B)
    vectorAdd_time = timer() - start

    print("c[:5] = " + str(C[:5]))
    print("c[-5:] = " + str(C[-5:]))

    print("vectorAdd took %f seconds " % vectorAdd_time)

if __name__ == '__main__':
    main()

In [ ]:
!pip install numba

In [ ]:
!lspci

In [ ]:
Ubuntu

lspci|grep - nvidia 即可看到当前显卡型号
nvidia-settings nvidia-prime双显卡切换
nvidia-settings 弹出nvidia信息窗口，可以看到驱动版本，GPU编号等
sudo dpkg --list | grep nvidia-*

NVIDIA Driver Version: 384.130

In [ ]:
!dpkg --list | grep nvidia-*

### ubuntu查看nvidia显卡状态
nvidia-smi

连续查看显卡状态
sudo watch -n 10 nvidia-smi

第一栏的Fan：N/A是风扇转速，从0到100%之间变动，这个速度是计算机期望的风扇转速，实际情况下如果风扇堵转，可能打不到显示的转速。有的设备不会返回转速，因为它不依赖风扇冷却而是通过其他外设保持低温（比如我们实验室的服务器是常年放在空调房间里的）。 

第二栏的Temp：是温度，单位摄氏度。 
第三栏的Perf：是性能状态，从P0到P12，P0表示最大性能，P12表示状态最小性能。 
第四栏下方的Pwr：是能耗，上方的Persistence-M：是持续模式的状态，持续模式虽然耗能大，但是在新的GPU应用启动时，花费的时间更少，这里显示的是off的状态。 
第五栏的Bus-Id是涉及GPU总线的东西，domain:bus:device.function 
第六栏的Disp.A是Display Active，表示GPU的显示是否初始化。 
第五第六栏下方的Memory Usage是显存使用率。 
第七栏是浮动的GPU利用率。 
第八栏上方是关于ECC的东西。 
第八栏下方Compute M是计算模式。 
下面一张表示每个进程占用的显存使用率。